<!-- ABSTRACT -->

With this script, we evaluate the ensembles and check how well they perform. The result is a plot of the uncertainty of the model's predictions. However, it seems that the uncertainty is not very high.

In [ ]:
import os
import sys
import json
import joblib

import numpy as np
from tqdm import tqdm
from scipy import stats
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
plt.rcParams["font.family"] = "Times New Roman"

import torch

# Add the 'scripts' directory to Python Path
scripts_path=os.path.abspath(os.path.join(os.getcwd(), '..'))
if scripts_path not in sys.path:
    sys.path.append(scripts_path)

import evaluation.help_functions as hf
import evaluation.plot_functions as pf

from gnn.help_functions import mc_dropout_predict
from gnn.models.point_net_transf_gat import PointNetTransfGAT
from data_preprocessing.help_functions import highway_mapping

In [ ]:
# Get the absolute path to the project root
project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))

# Paths
runs_path = os.path.join(project_root, "data", "runs_01_2025")
ensembles = ["ensemble_1",
             "ensemble_2",
             "ensemble_3",
             "ensemble_4_lrz",
             "ensemble_5"]

# Dropout for Ensembles
use_dropout = [False,
               False,
               False,
               True,
               True]

districts = gpd.read_file(os.path.join(project_root, "data", "visualisation", "districts_paris.geojson"))
base_case_path = os.path.join(project_root, "data", "links_and_stats", "pop_1pct_basecase_average_output_links.geojson")
links_base_case = gpd.read_file(base_case_path, crs="EPSG:4326")

# GNN Parameters
point_net_conv_layer_structure_local_mlp="256"
point_net_conv_layer_structure_global_mlp = "512"
gat_conv_layer_structure = "128,256,512"
predict_mode_stats = False
in_channels = 5
out_channels = 1

point_net_conv_layer_structure_local_mlp = [int(x) for x in point_net_conv_layer_structure_local_mlp.split(',')]
point_net_conv_layer_structure_global_mlp = [int(x) for x in point_net_conv_layer_structure_global_mlp.split(',')]
gat_conv_layer_structure = [int(x) for x in gat_conv_layer_structure.split(',')]

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
loss_fct = torch.nn.MSELoss().to(dtype=torch.float32).to(device)

In [ ]:
# Load test set and scalers
# Same for each ensemble with seed=42

ensemble_path = os.path.join(runs_path, ensembles[0])
data_created_during_training = os.path.join(ensemble_path, 'data_created_during_training/')

# Load scalers
scaler_x = joblib.load(data_created_during_training + 'test_x_scaler.pkl')
scaler_pos = joblib.load(data_created_during_training + 'test_pos_scaler.pkl')

# Load the test dataset created during training
test_set_dl = torch.load(data_created_during_training + 'test_dl.pt')

# Load the DataLoader parameters
with open(data_created_during_training + 'test_loader_params.json', 'r') as f:
    test_set_dl_loader_params = json.load(f)
    
# Remove or correct collate_fn if it is incorrectly specified
if 'collate_fn' in test_set_dl_loader_params and isinstance(test_set_dl_loader_params['collate_fn'], str):
    del test_set_dl_loader_params['collate_fn']  # Remove it to use the default collate function
    
test_set_loader = torch.utils.data.DataLoader(test_set_dl, **test_set_dl_loader_params)

In [ ]:
def load_ensemble(ensemble, model_path, use_dropout):

    model = PointNetTransfGAT(in_channels=in_channels, out_channels=out_channels,
            point_net_conv_layer_structure_local_mlp=point_net_conv_layer_structure_local_mlp, 
            point_net_conv_layer_structure_global_mlp = point_net_conv_layer_structure_global_mlp,
            gat_conv_layer_structure=gat_conv_layer_structure,
            use_dropout=use_dropout,
            predict_mode_stats=predict_mode_stats)
    
    model = model.to(device)
    model.load_state_dict(torch.load(model_path), strict=False)

    return model

In [ ]:
## Individual Ensembles Evaluation

for i, ensemble in enumerate(ensembles):

    model_path = os.path.join(runs_path, ensemble, 'trained_model/model.pth')
    model = load_ensemble(ensemble, model_path, use_dropout[i])
    
    test_loss, r_squared, actual_vals, predictions, baseline_loss = hf.validate_model_on_test_set(model, test_set_loader.dataset, loss_fct, device)

    print(f"\n{ensemble}:")
    print(f"Test Loss: {test_loss}")
    print(f"R-squared: {r_squared}")
    print(f"Baseline Loss: {baseline_loss}")

### Simple Analysis

In [ ]:
i = 0
print(f"Results for sample {i}\n")

test_data = test_set_loader.dataset[i]
test_x = test_set_loader.dataset[i].x
test_x = test_x.to('cpu')

for i, ensemble in enumerate(ensembles):

    print(f"Results for {ensemble}:")

    model_path = os.path.join(runs_path,ensemble,'trained_model/model.pth')
    model = load_ensemble(ensemble, model_path, use_dropout[i])

    test_loss, r_squared, actual_vals, predictions, baseline_loss = hf.validate_model_on_test_set(model, test_data, loss_fct, device)

    print(f"Test Loss: {test_loss}")
    print(f"R-squared: {r_squared}")
    print(f"Baseline Loss: {baseline_loss}")

    inversed_x = scaler_x.inverse_transform(test_x)

    gdf_with_og_values = hf.data_to_geodataframe_with_og_values(data=test_data, original_gdf=links_base_case, predicted_values=predictions, inversed_x=inversed_x, use_all_features=False)
    gdf_with_og_values['capacity_reduction_rounded'] = gdf_with_og_values['capacity_reduction'].round(decimals=3)
    gdf_with_og_values['highway'] = gdf_with_og_values['highway'].map(highway_mapping)

    # print("\nPredicted values:")
    # pf.plot_combined_output(gdf_input=gdf_with_og_values, column_to_plot="vol_car_change_predicted", 
    #                         save_it=False, number_to_plot=i, zone_to_plot = "this zone", is_predicted=True, alpha=0, 
    #                         use_fixed_norm=False, fixed_norm_max = fixed_norm_max,
    #                         known_districts = False, buffer = 0.0005, 
    #                         plot_policy_roads = True, 
    #                         plot_contour_lines = True, 
    #                         districts_of_interest=None)

    # print("Actual values:")
    # pf.plot_combined_output(gdf_input=gdf_with_og_values, column_to_plot="vol_car_change_actual", save_it=False, 
    #                         number_to_plot=i, zone_to_plot = "this zone",is_predicted=False,alpha=10,
    #                         use_fixed_norm=False, fixed_norm_max = fixed_norm_max,
    #                         known_districts = False, buffer = 0.0005, 
    #                         plot_policy_roads = True, 
    #                         plot_contour_lines = True, 
    #                         districts_of_interest=None)
    

In [ ]:
def ensemble_predict(model, data, device):
    
    predictions = []

    with torch.no_grad():
        for i, ensemble in enumerate(ensembles):
            
            model_path = os.path.join(runs_path, ensemble, 'trained_model/model.pth')
            model = load_ensemble(ensemble, model_path, use_dropout[i])
            model.eval()

            pred = model(data.to(device))
            if isinstance(pred, tuple):  # If multiple outputs (e.g., mode_stats)
                pred = pred[0]
            predictions.append(pred.cpu().numpy())  # Collect predictions

    # Stack predictions and calculate statistics
    predictions = np.stack(predictions, axis=0)  # Shape: (num_ensembles, num_predictions)
    mean_prediction = predictions.mean(axis=0)  # Mean prediction
    uncertainty = predictions.std(axis=0)       # Uncertainty (standard deviation)

    return mean_prediction, uncertainty

In [ ]:
# Ensemble uncertainty on single sample

i = 32
print(f"Ensemble uncertainty for sample {i}:")

test_data = test_set_loader.dataset[i]
test_x = test_set_loader.dataset[i].x
test_x = test_x.to('cpu')

inversed_x = scaler_x.inverse_transform(test_x)
mean_predictions, uncertainties = ensemble_predict(model, test_data, device=device)

gdf_with_og_values = hf.data_to_geodataframe_with_og_values(data=test_data, original_gdf=links_base_case, predicted_values=mean_predictions, inversed_x=inversed_x, use_all_features=False)
gdf_with_og_values['capacity_reduction_rounded'] = gdf_with_og_values['capacity_reduction'].round(decimals=3)
gdf_with_og_values['highway'] = gdf_with_og_values['highway'].map(highway_mapping)
gdf_with_og_values['ensemble_uncertainty'] = uncertainties

pf.plot_combined_output(gdf_input=gdf_with_og_values, column_to_plot="ensemble_uncertainty", plot_contour_lines=False,
                        save_it=False, number_to_plot=i, zone_to_plot = "this zone", is_predicted=True, use_fixed_norm=False,
                        known_districts = False, buffer = 0.0005, districts_of_interest=None, cmap='Reds')

In [ ]:
# Ensemble uncertainty on entire test set

mean_uncertainties = []

for i in tqdm.tqdm(range(len(test_set_loader.dataset))):
    
    test_data = test_set_loader.dataset[i]
    test_x = test_set_loader.dataset[i].x
    test_x = test_x.to('cpu')

    mean_predictions, uncertainties = ensemble_predict(model, test_data, device=device)
    mean_uncertainties.append(uncertainties)

mean_uncertainties = np.array(mean_uncertainties).mean(axis=0)

In [ ]:
# Some stats to pick discrete bins

flat_uc = np.sort(mean_uncertainties.flatten())

percentile_1 = np.percentile(flat_uc, 25)
percentile_2 = np.percentile(flat_uc, 75)

print(f"1st Percentile: {percentile_1}")
print(f"2nd Percentile: {percentile_2}")

In [ ]:
# On the last sample, but does not matter
inversed_x = scaler_x.inverse_transform(test_x)
gdf_with_og_values = hf.data_to_geodataframe_with_og_values(data=test_data, original_gdf=links_base_case, predicted_values=mean_predictions, inversed_x=inversed_x, use_all_features=False)
gdf_with_og_values['capacity_reduction_rounded'] = gdf_with_og_values['capacity_reduction'].round(decimals=3)
gdf_with_og_values['highway'] = gdf_with_og_values['highway'].map(highway_mapping)
gdf_with_og_values['ensemble_uncertainty'] = mean_uncertainties

dicrete_thresholds = [1,2]

pf.plot_combined_output(gdf_input=gdf_with_og_values, column_to_plot="ensemble_uncertainty", plot_contour_lines=False,
                        save_it=False, number_to_plot=i+1, zone_to_plot = "this zone", is_predicted=True, use_fixed_norm=False,
                        known_districts = False, buffer = 0.0005, districts_of_interest=None, scale_type='discrete', discrete_thresholds=dicrete_thresholds)

### Bias Variance Analysis

In [ ]:
def get_ensemble_predictions_on_test_set(test_set_loader, device):
    """Get raw predictions from all ensemble members"""
    all_predictions = []
    
    for i, ensemble in (enumerate(tqdm(ensembles))):
        
        model_path = os.path.join(runs_path, ensemble, 'trained_model/model.pth')
        model = load_ensemble(ensemble, model_path, use_dropout[i])

        test_loss, r_squared, actual_vals, predictions, baseline_loss = hf.validate_model_on_test_set(model, test_set_loader.dataset, loss_fct, device)
        all_predictions.append(predictions.view(predictions.shape[0]//31635, 31635).cpu().numpy())
    
    return np.array(all_predictions), np.array(actual_vals.view(actual_vals.shape[0]//31635, 31635).cpu().numpy())

all_predictions, all_actuals = get_ensemble_predictions_on_test_set(test_set_loader, device)

In [ ]:
def compute_bias_variance(actual_values, ensemble_predictions):
    """
    Compute bias and variance from ensemble predictions
    
    Parameters:
    - actual_values: numpy array of true values
    - ensemble_predictions: numpy array of shape (n_ensembles, n_samples)
    
    Returns:
    - bias: average bias (squared difference between mean prediction and true values)
    - variance: average variance of predictions
    """
    mean_predictions = np.mean(ensemble_predictions, axis=0)
    
    # Compute bias (squared difference between mean prediction and true value)
    bias = np.mean((mean_predictions - actual_values) ** 2)
    
    # Compute variance (average variance of predictions across ensemble members)
    variance = np.mean(np.var(ensemble_predictions, axis=0))
    
    return bias, variance

# Overall bias-variance
overall_bias, overall_variance = compute_bias_variance(all_actuals, all_predictions)
print(f"\nOverall Metrics:")
print(f"Bias: {overall_bias:.4f}")
print(f"Variance: {overall_variance:.4f}")
print(f"Total Error (Bias + Variance): {overall_bias + overall_variance:.4f}")

In [ ]:
simple_highway_mapping = {
    'trunk': 0,
    'primary': 1,
    'secondary': 2,
    'tertiary': 3,
    'residential': 4, 'living_street': 5,
    'pedestrian': 6, 'service': 7,
    'construction': 8, 'unclassified': 9,
    'all_types': 'all types'
}

highway_descriptions = {
    'trunk': 'Trunk Roads',
    'primary': 'Primary Roads',
    'secondary': 'Secondary Roads',
    'tertiary': 'Tertiary Roads',
    'living_street': 'Living Streets',
    'residential': 'Residential Streets'
}

In [ ]:
print("Bias-Variance by Road Type:")
for highway_type, code in simple_highway_mapping.items():
    if highway_type == 'all_types':
        continue
    
    # Get mask for this road type
    road_mask = gdf_with_og_values['highway'] == code
    if not road_mask.any():
        continue
    
    # Get predictions for this road type
    road_predictions = all_predictions[:, :, road_mask]  # Shape: (5, 831, n_roads_of_type)
    road_predictions = road_predictions.reshape(5, -1)  # Shape: (5, 831 * n_roads_of_type)
    
    # Handle actuals - first reshape to remove last dimension of 1, then apply mask
    actuals_reshaped = all_actuals.reshape(831, -1)  # Shape: (831, 31635)
    road_actuals = actuals_reshaped[:, road_mask].reshape(-1)  # Shape: (831 * n_roads_of_type)
    
    # Compute metrics
    road_bias, road_variance = compute_bias_variance(road_actuals, road_predictions)
    print(f"\n{highway_type}:")
    print(f"Bias: {road_bias:.4f}")
    print(f"Variance: {road_variance:.4f}")
    print(f"Total Error: {road_bias + road_variance:.4f}")

In [ ]:
# Road types we're interested in
road_types_of_interest = ['primary', 'secondary', 'tertiary']

# Create gdf_with_og_values, just for having the dataframe.
i = 0
test_data = test_set_loader.dataset[i]
test_x = test_set_loader.dataset[i].x
test_x = test_x.to('cpu')
links_base_case = gpd.read_file(base_case_path, crs="EPSG:4326")

for i, ensemble in enumerate(ensembles):
    model_path = os.path.join(runs_path,ensemble,'trained_model/model.pth')
    model = load_ensemble(ensemble, model_path, use_dropout[i])
    inversed_x = scaler_x.inverse_transform(test_x)
    gdf_with_og_values = hf.data_to_geodataframe_with_og_values(data=test_data, original_gdf=links_base_case, predicted_values=predictions, inversed_x=inversed_x, use_all_features=False)
    gdf_with_og_values['capacity_reduction_rounded'] = gdf_with_og_values['capacity_reduction'].round(decimals=3)
    gdf_with_og_values['highway'] = gdf_with_og_values['highway'].map(highway_mapping)
    break

print("\nBias-Variance by Road Type and Capacity Reduction:")
for highway_type in road_types_of_interest:
    description = highway_descriptions[highway_type]
    
    # Base mask for road type
    road_type_mask = gdf_with_og_values['highway'] == highway_mapping[highway_type]
    
    # Create masks for with and without capacity reduction
    with_reduction_mask = road_type_mask & (gdf_with_og_values['capacity_reduction_rounded'] < 0)
    without_reduction_mask = road_type_mask & (gdf_with_og_values['capacity_reduction_rounded'] == 0)
    
    for mask, condition in [(with_reduction_mask, "with capacity reduction"), 
                           (without_reduction_mask, "without capacity reduction")]:
        if not mask.any():
            print(f"\n{description} {condition}: No roads found")
            continue
            
        # Get predictions for this combination
        road_predictions = all_predictions[:, :, mask]
        road_predictions = road_predictions.reshape(5, -1)
        
        # Handle actuals
        actuals_reshaped = all_actuals.reshape(831, -1)
        road_actuals = actuals_reshaped[:, mask].reshape(-1)
        
        # Compute metrics
        road_bias, road_variance = compute_bias_variance(road_actuals, road_predictions)
        print(f"\n{description} {condition}:")
        print(f"Number of road segments: {mask.sum()}")
        print(f"Bias: {road_bias:.4f}")
        print(f"Variance: {road_variance:.4f}")
        print(f"Total Error: {road_bias + road_variance:.4f}")

In [ ]:
### Across all ensembles ###

# Road types we're interested in
road_types_of_interest = ['primary', 'secondary', 'tertiary']

# Create gdf_with_og_values with averaged predictions across all ensembles and all test samples
test_x = test_set_loader.dataset[0].x  # Just to get inversed_x for the geodataframe
test_x = test_x.to('cpu')
links_base_case = gpd.read_file(base_case_path, crs="EPSG:4326")
inversed_x = scaler_x.inverse_transform(test_x)

# Average predictions across all test samples and ensembles
mean_predictions = np.mean(all_predictions, axis=(0, 1))  # Average over samples (831) and ensembles (5)

# Create geodataframe with mean predictions
gdf_with_og_values = hf.data_to_geodataframe_with_og_values(data=test_set_loader.dataset[0], 
                                                           original_gdf=links_base_case, 
                                                           predicted_values=mean_predictions, 
                                                           inversed_x=inversed_x, 
                                                           use_all_features=False)
gdf_with_og_values['capacity_reduction_rounded'] = gdf_with_og_values['capacity_reduction'].round(decimals=3)
gdf_with_og_values['highway'] = gdf_with_og_values['highway'].map(highway_mapping)

print("\nBias-Variance by Road Type and Capacity Reduction:")
for highway_type in road_types_of_interest:
    description = highway_descriptions[highway_type]
    
    # Base mask for road type
    road_type_mask = gdf_with_og_values['highway'] == highway_mapping[highway_type]
    
    # Create masks for with and without capacity reduction
    with_reduction_mask = road_type_mask & (gdf_with_og_values['capacity_reduction_rounded'] < 0)
    without_reduction_mask = road_type_mask & (gdf_with_og_values['capacity_reduction_rounded'] == 0)
    
    for mask, condition in [(with_reduction_mask, "with capacity reduction"), 
                           (without_reduction_mask, "without capacity reduction")]:
        if not mask.any():
            print(f"\n{description} {condition}: No roads found")
            continue
            
        # Get predictions for this combination
        road_predictions = all_predictions[:, :, mask]
        road_predictions = road_predictions.reshape(5, -1)
        
        # Handle actuals
        actuals_reshaped = all_actuals.reshape(831, -1)
        road_actuals = actuals_reshaped[:, mask].reshape(-1)
        
        # Compute metrics
        road_bias, road_variance = compute_bias_variance(road_actuals, road_predictions)
        print(f"\n{description} {condition}:")
        print(f"Number of road segments: {mask.sum()}")
        print(f"Bias: {road_bias:.4f}")
        print(f"Variance: {road_variance:.4f}")
        print(f"Total Error: {road_bias + road_variance:.4f}")

In [ ]:
# Define road types
basic_road_types = ['trunk', 'primary', 'secondary', 'tertiary', 'living_street', 'residential']
pst_road_types = ['primary', 'secondary', 'tertiary']

# Collect results in a structured format
results = []

# First, add all basic road types (without distinguishing capacity reduction)
for highway_type in basic_road_types:
    description = highway_descriptions[highway_type]
    
    # Base mask for road type
    road_type_mask = gdf_with_og_values['highway'] == highway_mapping[highway_type]
    if not road_type_mask.any():
        continue
        
    road_predictions = all_predictions[:, :, road_type_mask]
    road_predictions = road_predictions.reshape(5, -1)
    actuals_reshaped = all_actuals.reshape(831, -1)
    road_actuals = actuals_reshaped[:, road_type_mask].reshape(-1)
    
    road_bias, road_variance = compute_bias_variance(road_actuals, road_predictions)
    
    results.append({
        'Road Category': description,
        'Bias': road_bias,
        'Surrogate Model Variance': road_variance,
        'Total Error': road_bias + road_variance
    })

# Now add PST roads with and without capacity reduction
pst_mask = np.zeros(len(gdf_with_og_values), dtype=bool)
for highway_type in pst_road_types:
    pst_mask |= (gdf_with_og_values['highway'] == highway_mapping[highway_type])

# With capacity reduction
with_reduction_mask = pst_mask & (gdf_with_og_values['capacity_reduction_rounded'] < 0)
road_predictions = all_predictions[:, :, with_reduction_mask]
road_predictions = road_predictions.reshape(5, -1)
actuals_reshaped = all_actuals.reshape(831, -1)
road_actuals = actuals_reshaped[:, with_reduction_mask].reshape(-1)
road_bias, road_variance = compute_bias_variance(road_actuals, road_predictions)
results.append({
    'Road Category': 'P/S/T Roads with capacity reduction',
    'Bias': road_bias,
    'Surrogate Model Variance': road_variance,
    'Total Error': road_bias + road_variance
})

# Without capacity reduction
without_reduction_mask = pst_mask & (gdf_with_og_values['capacity_reduction_rounded'] == 0)
road_predictions = all_predictions[:, :, without_reduction_mask]
road_predictions = road_predictions.reshape(5, -1)
actuals_reshaped = all_actuals.reshape(831, -1)
road_actuals = actuals_reshaped[:, without_reduction_mask].reshape(-1)
road_bias, road_variance = compute_bias_variance(road_actuals, road_predictions)
results.append({
    'Road Category': 'P/S/T Roads without capacity reduction',
    'Bias': road_bias,
    'Surrogate Model Variance': road_variance,
    'Total Error': road_bias + road_variance
})

# Convert to DataFrame for easier plotting
df_results = pd.DataFrame(results)

# Calculate Pearson correlation coefficient
correlation, p_value = stats.pearsonr(df_results['Bias'], df_results['Surrogate Model Variance'])

# plt.style.use('default')

# Create scatter plot
plt.figure(figsize=(10, 8))

plt.grid(False)
ax = plt.gca()
ax.set_axisbelow(False)
ax.spines['top'].set_visible(True)
ax.spines['right'].set_visible(True)

# Plot points
scatter = plt.scatter(df_results['Bias'], 
                     df_results['Surrogate Model Variance'],
                     c=[plt.cm.Set2(i) for i in range(len(df_results))],
                     s=100)

# Add trend line
z = np.polyfit(df_results['Bias'], df_results['Surrogate Model Variance'], 1)
p = np.poly1d(z)
x_trend = np.linspace(df_results['Bias'].min(), df_results['Bias'].max(), 100)
plt.plot(x_trend, p(x_trend), "gray", linestyle="--", alpha=0.7)

# Add correlation coefficient to plot
plt.text(0.05, 0.95, f'Pearson r = {correlation:.2f}', 
         transform=plt.gca().transAxes, 
         fontsize=12)

# Add labels for each point
for idx, row in df_results.iterrows():
    plt.annotate(row['Road Category'], 
                (row['Bias'], row['Surrogate Model Variance']),
                xytext=(5, 5), 
                textcoords='offset points',
                fontsize=12)

plt.xlabel('Bias', fontsize=12)
plt.ylabel('Surrogate Model Variance', fontsize=12)
# plt.title('Bias-Variance Trade-off by Road Category', fontsize=12)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

# Set axes to start at 0
plt.xlim(left=0)
plt.ylim(bottom=0)

plt.tight_layout()
plt.savefig("bias_variance_tradeoff.png", dpi = 500)
plt.show()

### Plotting Uncertainty

In [ ]:
all_predictions, all_actuals = get_ensemble_predictions_on_test_set(test_set_loader, device)

# Standard deviation across ensemble members
mean_uncertainties = np.std(all_predictions, axis=0)

# Average uncertainties across all test samples
mean_uncertainties = np.array(mean_uncertainties).mean(axis=0)

# Create visualization
flat_uc = np.sort(mean_uncertainties.flatten())
percentile_25 = np.percentile(flat_uc, 25)
percentile_75 = np.percentile(flat_uc, 75)

print(f"25th Percentile: {percentile_25:.4f}")
print(f"75th Percentile: {percentile_75:.4f}")
print(f"Mean uncertainty: {np.mean(flat_uc):.4f}")
print(f"Max uncertainty: {np.max(flat_uc):.4f}")

# Add uncertainties to the GeoDataFrame
gdf_with_og_values['ensemble_uncertainty'] = mean_uncertainties

# Plot with discrete color levels
discrete_thresholds = [percentile_25, percentile_75]

pf.plot_combined_output(
    gdf_input=gdf_with_og_values, 
    column_to_plot="ensemble_uncertainty",
    plot_contour_lines=False,
    save_it=False, 
    number_to_plot=1,
    zone_to_plot="this zone",
    is_predicted=True,
    use_fixed_norm=False,
    known_districts=False,
    buffer=0.0005,
    districts_of_interest=None,
    scale_type='discrete',
    discrete_thresholds=discrete_thresholds,
    cmap='Reds'
)

In [ ]:
def get_ensemble_predictions(data, device):
    """Get raw predictions from all ensemble members"""
    predictions = []
    
    for i, ensemble in enumerate(ensembles):
        model_path = os.path.join(runs_path, ensemble, 'trained_model/model.pth')
        model = load_ensemble(ensemble, model_path, use_dropout[i])
        model.eval()
        
        with torch.no_grad():
            pred = model(data.to(device))
            pred = pred.cpu().numpy()
            predictions.append(pred)

            # # Use the existing mc_dropout_predict function with num_samples=1
            # pred = mc_dropout_predict(model, data, num_samples=1, device=device)
            # if isinstance(pred, tuple):
            #     pred = pred[0]
            # predictions.append(pred)
    
    return np.array(predictions)

In [ ]:
# Let's look at some example predictions and their relative uncertainties
test_data = test_set_loader.dataset[0]
ensemble_preds = get_ensemble_predictions(test_data, device)

print("Shape of ensemble predictions:", ensemble_preds.shape)
print("\nExample predictions for first 3 road segments:")
for i in range(3):
    road_preds = ensemble_preds[:, i, 0]  # predictions from all ensembles for road i
    mean_pred = np.mean(road_preds)
    abs_uncertainty = np.std(road_preds)
    rel_uncertainty = (abs_uncertainty / mean_pred) * 100  # as percentage
    print(f"\nRoad {i}:")
    print(f"Predictions from ensembles: {road_preds}")
    print(f"Mean prediction: {mean_pred:.3f}")
    print(f"Absolute uncertainty (std): {abs_uncertainty:.3f}")
    print(f"Relative uncertainty (CV): {rel_uncertainty:.1f}%")

In [ ]:
# Now compute relative uncertainties for all roads
mean_relative_uncertainties = []

print("\nComputing relative uncertainties across test set...")
for i in tqdm(range(len(test_set_loader.dataset))):
    test_data = test_set_loader.dataset[i]
    ensemble_preds = get_ensemble_predictions(test_data, device)
    
    # Calculate relative uncertainty (CV) for each road
    std_devs = np.std(ensemble_preds, axis=0)
    means = np.mean(ensemble_preds, axis=0)
    relative_uncertainties = (std_devs / means) * 100  # as percentage
    mean_relative_uncertainties.append(relative_uncertainties)

mean_relative_uncertainties = np.array(mean_relative_uncertainties).mean(axis=0)

# Create visualization
flat_uc = np.sort(mean_relative_uncertainties.flatten())
percentile_25 = np.percentile(flat_uc, 25)
percentile_75 = np.percentile(flat_uc, 75)

print("\nRelative Uncertainty Statistics:")
print(f"25th Percentile: {percentile_25:.1f}%")
print(f"75th Percentile: {percentile_75:.1f}%")
print(f"Mean relative uncertainty: {np.mean(flat_uc):.1f}%")
print(f"Max relative uncertainty: {np.max(flat_uc):.1f}%")

# Add relative uncertainties to the GeoDataFrame
gdf_with_og_values['ensemble_uncertainty'] = mean_relative_uncertainties

# Plot with discrete color levels
discrete_thresholds = [percentile_25, percentile_75]

pf.plot_combined_output(
    gdf_input=gdf_with_og_values, 
    column_to_plot="ensemble_uncertainty",
    plot_contour_lines=False,
    save_it=False, 
    number_to_plot=1,
    zone_to_plot="this zone",
    is_predicted=True,
    use_fixed_norm=False,
    known_districts=False,
    buffer=0.0005,
    districts_of_interest=None,
    scale_type='discrete',
    discrete_thresholds=discrete_thresholds,
    cmap='Reds'
)

In [ ]:
# Now compute uncertainties using a more robust measure
mean_uncertainties = []

print("\nComputing uncertainties across test set...")
for i in tqdm(range(len(test_set_loader.dataset))):
    test_data = test_set_loader.dataset[i]
    ensemble_preds = get_ensemble_predictions(test_data, device)
    
    # Calculate uncertainty as the range of predictions
    # (max - min) / (|mean| + epsilon) to handle zeros and negative values
    max_preds = np.max(ensemble_preds, axis=0)
    min_preds = np.min(ensemble_preds, axis=0)
    mean_preds = np.mean(ensemble_preds, axis=0)
    epsilon = 1.0  # small constant to avoid division by zero and handle small values
    
    uncertainty = (max_preds - min_preds) / (np.abs(mean_preds) + epsilon)
    mean_uncertainties.append(uncertainty)

mean_uncertainties = np.array(mean_uncertainties).mean(axis=0)

# Print some statistics
print("\nUncertainty Statistics:")
print(f"Mean uncertainty: {np.mean(mean_uncertainties):.3f}")
print(f"Max uncertainty: {np.max(mean_uncertainties):.3f}")
print(f"Min uncertainty: {np.min(mean_uncertainties):.3f}")

# Add uncertainties to the GeoDataFrame
gdf_with_og_values['ensemble_uncertainty'] = mean_uncertainties

# Plot with discrete color levels
percentile_25 = np.percentile(mean_uncertainties, 25)
percentile_75 = np.percentile(mean_uncertainties, 75)
discrete_thresholds = [percentile_25, percentile_75]

pf.plot_combined_output(
    gdf_input=gdf_with_og_values, 
    column_to_plot="ensemble_uncertainty",
    plot_contour_lines=False,
    save_it=False, 
    number_to_plot=1,
    zone_to_plot="this zone",
    is_predicted=True,
    use_fixed_norm=False,
    known_districts=False,
    buffer=0.0005,
    districts_of_interest=None,
    scale_type='discrete',
    discrete_thresholds=discrete_thresholds,
    cmap='Reds'
)